In [45]:
%load_ext autoreload
%autoreload 2

import spglib as spg
import ase
from ase.visualize import view

import ase.spacegroup as aspg
import numpy as np
import pandas as pd
from collections import Iterable
import cPickle as pck

import sys, os
# add to PYTHONPATH
modulepath = '/home/musil/Dropbox/Felix_work/qmat/spglib-test/'
#modulepath = '/Users/iMac/Dropbox/Felix_work/qmat/spglib-test/'
#if modulepath not in sys.path:
#    sys.path.insert(0,modulepath)

#from genGeneralSites import GenCrystalStructures
from qe_input import makeCard,makeNamelist,SG2ibrav,SG2wyckoff,makeQEInnput_sg,makeQEInnput_ibrav0
from utils import rotation_matrix

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### QE determination of ibrav

1          cubic P (sc)
      v1 = a(1,0,0),  v2 = a(0,1,0),  v3 = a(0,0,1)

  2          cubic F (fcc)
      v1 = (a/2)(-1,0,1),  v2 = (a/2)(0,1,1), v3 = (a/2)(-1,1,0)

  3          cubic I (bcc)
      v1 = (a/2)(1,1,1),  v2 = (a/2)(-1,1,1),  v3 = (a/2)(-1,-1,1)

  4          Hexagonal and Trigonal P        celldm(3)=c/a
      v1 = a(1,0,0),  v2 = a(-1/2,sqrt(3)/2,0),  v3 = a(0,0,c/a)

  5          Trigonal R, 3fold axis c        celldm(4)=cos(alpha)
      The crystallographic vectors form a three-fold star around
      the z-axis, the primitive cell is a simple rhombohedron:
      v1 = a(tx,-ty,tz),   v2 = a(0,2ty,tz),   v3 = a(-tx,-ty,tz)
      where c=cos(alpha) is the cosine of the angle alpha between
      any pair of crystallographic vectors, tx, ty, tz are:
        tx=sqrt((1-c)/2), ty=sqrt((1-c)/6), tz=sqrt((1+2c)/3)


  6          Tetragonal P (st)               celldm(3)=c/a
      v1 = a(1,0,0),  v2 = a(0,1,0),  v3 = a(0,0,c/a)

  7          Tetragonal I (bct)              celldm(3)=c/a
      v1=(a/2)(1,-1,c/a),  v2=(a/2)(1,1,c/a),  v3=(a/2)(-1,-1,c/a)

  8          Orthorhombic P                  celldm(2)=b/a
                                             celldm(3)=c/a
      v1 = (a,0,0),  v2 = (0,b,0), v3 = (0,0,c)

  9          Orthorhombic base-centered(bco) celldm(2)=b/a
                                             celldm(3)=c/a
      v1 = (a/2, b/2,0),  v2 = (-a/2,b/2,0),  v3 = (0,0,c)
 -9          as 9, alternate description
      v1 = (a/2,-b/2,0),  v2 = (a/2, b/2,0),  v3 = (0,0,c)

 10          Orthorhombic face-centered      celldm(2)=b/a
                                             celldm(3)=c/a
      v1 = (a/2,0,c/2),  v2 = (a/2,b/2,0),  v3 = (0,b/2,c/2)

 11          Orthorhombic body-centered      celldm(2)=b/a
                                             celldm(3)=c/a
      v1=(a/2,b/2,c/2),  v2=(-a/2,b/2,c/2),  v3=(-a/2,-b/2,c/2)

 12          Monoclinic P, unique axis c     celldm(2)=b/a
                                             celldm(3)=c/a,
                                             celldm(4)=cos(ab)
      v1=(a,0,0), v2=(b*cos(gamma),b*sin(gamma),0),  v3 = (0,0,c)
      where gamma is the angle between axis a and b.
-12          Monoclinic P, unique axis b     celldm(2)=b/a
                                             celldm(3)=c/a,
                                             celldm(5)=cos(ac)
      v1 = (a,0,0), v2 = (0,b,0), v3 = (c*cos(beta),0,c*sin(beta))
      where beta is the angle between axis a and c

 13          Monoclinic base-centered        celldm(2)=b/a
                                             celldm(3)=c/a,
                                             celldm(4)=cos(ab)
      v1 = (  a/2,         0,                -c/2),
      v2 = (b*cos(gamma), b*sin(gamma), 0),
      v3 = (  a/2,         0,                  c/2),
      where gamma is the angle between axis a and b

 14          Triclinic

## Load general information

In [46]:
fileNames = {}
infoPath = '../info/'
structurePath = '../structures/'
fileNames['crystals'] = structurePath + 'partial_input_crystals_sg3-230.pck'
fileNames['wyck'] = infoPath+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = infoPath+'SpaceGroup-general-info.pck'
fileNames['elements info'] = infoPath+'General-Info-Elements-fast.pck'

In [47]:
with open(fileNames['crystals'],'rb') as f:
    crystals = pck.load(f)
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

In [4]:
inputDic = {}

In [7]:
sgIdx = 100
cc = crystals[sgIdx][0]

inputDic[sgIdx] = makeQEInnput_sg(cc,sgIdx,WyckTable,SGTable,ElemTable,
                                    zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,
                                 calculation_type='"scf"',
                             kpt = [2,2,2],kpt_offset = [0,0,0],ppPath='"./pseudo/"',
                             PP=['Si.pbe-n-rrkjus_psl.1.0.0.UPF'])
print inputDic[sgIdx]

&CONTROL 
  wf_collect = .false. 
  calculation = "scf" 
  verbosity = "high" 
  pseudo_dir = "../pseudo/" 
  prefix = "qe" 
  restart_mode = "from_scratch" 
  outdir = "./out/" 
/ 
&SYSTEM 
  ecutrho = 80.00000 
  ecutwfc = 20.00000 
  ibrav = 6 
  nat = 1 
  nbnd = 20 
  ntyp = 1 
  occupations = "smearing" 
  smearing = "cold" 
  degauss = 1.0000000000d-02 
  space_group = 100 
  A = 7.647657865 
  B = 7.647657865 
  C = 2.24430468277 
  cosAB = 0.00000 
  cosAC = 0.00000 
  cosBC = 0.00000 
/ 
&ELECTRONS 
  conv_thr = 1.0000000000d-06 
/ 
ATOMIC_SPECIES   
  Si  28.0855  Si.pbe-n-rrkjus_psl.1.0.0.UPF 
ATOMIC_POSITIONS  crystal_sg 
  Si  8d  0.64759 
          0.645009 
          0.49 
K_POINTS  automatic 
  2  2  2  0  0  0 



In [36]:
def make_dir(file_path):
    directory = os.path.abspath(file_path)
    suffix = 0
    while os.path.exists(directory+'-{}'.format(suffix)):
        suffix += 1
    path = directory +'-{}/'.format(suffix)
    os.makedirs(path)
    return path
    

In [37]:
dirNames = {sg:'../test_run/data_1/sg-test_'+str(sg) for sg in [100]}

In [38]:
import subprocess as sp
os.environ['TMPDIR'] = "/tmp/"

In [50]:
exitState = {}
errors = {}
#path2pw = '/home/musil/source/qe-6.1/bin/'
path2pw = '/Users/iMac/source/qe-6.0/bin/'
for sg in dirNames:
    path = make_dir(dirNames[sg])
    inputName = os.path.abspath(path+'/qe.in')
    outputName = os.path.abspath(path+'/qe.out')
    errName = os.path.abspath(path+'/qe.err')
    with open(inputName,'w') as f:
        f.write(inputDic[sg])
    param = '-in '+ inputName
    print param
    # Set up the echo command and direct the output to a pipe
    #exitState[sg] = sp.call('mpirun -np 2 '+path2pw+'pw.x ' + param , stdout=open(outputName, 'w'), 
    #                         stderr=open(errName, 'w'), shell=True)
    exitState[sg] = sp.call('/opt/openmpi/bin/mpirun '+path2pw+'pw.x ' + param , stdout=open(outputName, 'w'), 
                             stderr=open(errName, 'w'), shell=True)
    if exitState[sg]:
        print sg
        with open(outputName,'r') as f:
            errors[sg] = f.read()

-in /Users/iMac/Dropbox/Felix_work/qmat/run_qe/test_run/data_1/sg-test_100-1/qe.in
100


In [49]:
print exitState[sg]
print errors[100]


126

